In [309]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [310]:
# import
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.tree import DecisionTreeRegressor
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

In [311]:
# load data
data = pd.read_csv('/kaggle/input/amsterdam-house-price-prediction/HousingPrices-Amsterdam-August-2021.csv', index_col=0)

In [312]:
print(data.head())
print(data.shape)

In [313]:
# check nan
data.isna().sum()

In [314]:
# fill nan by interpolate
data = data.dropna()

In [315]:
data.isna().sum()

In [316]:
data.describe()

In [317]:
# before remove outliers
sns.boxplot(x='Price', data = data)

In [318]:
# remove outliers (IQR)
q1 = data.describe()['Price']['25%']
q3 = data.describe()['Price']['75%']
iqr = q3 - q1
max_price = q3 + 1.5 * iqr

In [319]:
# calculate percens of 
outliers = data[data['Price'] >= max_price]
outliers_count = outliers['Price'].count()
prices_count = data['Price'].count()
print('Percentage removed: ' + str(round(outliers_count/prices_count * 100, 2)) + '%')

# remove outliers
data = data[data['Price'] < max_price]

In [320]:
# after remove outliers
sns.boxplot(x='Price', data = data)

In [321]:
# get street from address
def get_street(address):
    list = address.split()
    word = []
    for element in list:
        if element.isalpha() == True: 
            word.append(element)
        else:
            break
    word = ' '.join(word)
    return word

In [322]:
tqdm.pandas()
data['Zip No'] = data['Zip'].progress_apply(lambda x: x.split()[0])
data['Zip Code'] = data['Zip'].progress_apply(lambda x: x.split()[-1])
data['Address'] = data['Address'].progress_apply(lambda x: x.split(',')[0])
data['Street'] = data['Address'].progress_apply(lambda x: get_street(x))

In [323]:
data.head()

In [324]:
# drop unnecessary columns
data = data.drop(columns=['Address', 'Zip'])

In [325]:
# price / area
data['Price_m2'] = data['Price'] / data['Area']

In [326]:
data.head()

In [327]:
# encoder string
lbe = LabelEncoder()
lbe.fit(list(data['Zip Code'].values))
data['Zip Code'] = lbe.transform(list(data['Zip Code'].values))
lbe.fit(list(data['Street'].values))
data['Street'] = lbe.transform(list(data['Street'].values))

In [328]:
data.head()

In [329]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.distplot(data['Price'], bins=30)
plt.show()

In [330]:
sns.pairplot(data, x_vars=['Area', 'Room', 'Lon', 'Lat', 'Zip Code', 'Zip No', 'Street', 'Price_m2'], y_vars='Price', height=4, aspect=1, kind='scatter')
plt.show()

In [331]:
sns.pairplot(data)

In [332]:
sns.heatmap(data.corr(), annot = True)
plt.show()

In [333]:
# get X and y
X = data.drop(columns=['Price', 'Room', 'Lon', 'Lat', 'Zip Code', 'Zip No', 'Street'])
y = data.Price

In [334]:
X

In [335]:
# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, random_state=1008)

In [336]:
# standard data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [337]:
# custome
def linear_regression(X, y, eta, loop_number):
    X = np.asarray(X)
    m = X.shape[0]
    n = X.shape[1]
    
    # initial theta
    theta = list(np.random.rand(n + 1))
    y = np.asarray(y)
    len_theta = len(theta)
    
    # loop for number
    for k in range(loop_number):
#         print(f'Loop {k + 1}:')
        
        for i in range(m):
            h_i = 0
            row = np.append(1, X[i])
            
            # cal h_i
            for j in range(len_theta):
                h_i += theta[j] * row[j]
            
            # cal theta
            for j in range(len_theta):
#                 print(f'theta {j} = {theta[j]} + {eta} * ({y[i]} - {h_i}) * {row[j]} = {theta[j] + eta * (y[i] - h_i) * row[j]}')
                theta[j] = theta[j] + eta * (y[i] - h_i) * row[j]
            
#             print(theta)
    return theta


def predict_lm(X, theta):
    X = np.asarray(X)
    m = X.shape[0]
    n = X.shape[1]
    y_pred = []
    
    for i in range(m):
        pred = 0
        row = np.append(1, X[i])
        for j in range(n + 1):
            pred += theta[j] * row[j]
        
        y_pred.append(pred)
    
    return y_pred

In [338]:
# prediction
theta = linear_regression(X_train, y_train, 0.01, 10)
y_pred = predict_lm(X_test, theta)
print("Mean_squared_error: ", np.sqrt(mean_squared_error(y_test, y_pred)))
print('Coefficient of determination: %.2f' % r2_score(y_test, y_pred))

In [339]:
plt.scatter(y_test, y_pred)
plt.xlabel("Prices: $Y_i$")
plt.ylabel("Predicted prices")
plt.title("Prices vs Predicted prices")